In [1]:
!pip install flask==0.12.2
!pip install flask-ngrok
!pip install Jinja2

  Using cached Flask-0.12.2-py2.py3-none-any.whl (83 kB)
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [2]:
from flask import Flask, request, render_template_string
import threading
import pandas as pd


In [3]:
# import pandas as pd
# import random
# from flask import Flask, render_template_string, request, redirect, url_for
# from threading import Thread

# # Create the Flask application
# app = Flask(__name__)

# # Assume 'champion_name.csv' is your file containing champion names
# df = pd.read_csv('League of Legends Champion Stats 13.3.csv', sep=';')
# unique_champions_list = df['Name'].unique().tolist()

# def calculation(blue_champion, position1, position2, rank): 
#     return ['Lucian', 'Nami']

# # Updated template with additional form fields
# template = """
# <!DOCTYPE html>
# <html>
# <head>
#     <title>Select Your Champions</title>
# </head>
# <body>
#     <div>
#         <h2>Blue Side (Randomly Selected)</h2>
#         <p>{{ blue_champion }}</p>
#     </div>

#     <div>
#         <h2>Red Side (Select Two Positions and Your Rank)</h2>
#         <form method="POST">
#             <select name="position1">
#                 {% for position in positions %}
#                     <option value="{{ position }}">{{ position }}</option>
#                 {% endfor %}
#             </select>
#             <select name="position2">
#                 {% for position in remaining_positions %}
#                     <option value="{{ position }}">{{ position }}</option>
#                 {% endfor %}
#             </select>
#             <select name="rank">
#                 {% for rank in ranks %}
#                     <option value="{{ rank }}">{{ rank }}</option>
#                 {% endfor %}
#             </select>
#             <input type="submit" value="Submit">
#         </form>
#     </div>

#     {% if recommended_champions %}
#     <div>
#         <h2>Recommended Champions</h2>
#         <ul>
#             {% for champion in recommended_champions %}
#                 <li>{{ champion }}</li>
#             {% endfor %}
#         </ul>
#     </div>
#     {% endif %}
# </body>
# </html>
# """

# @app.route('/', methods=['GET', 'POST'])
# def home():
#     # Define positions and ranks
#     positions = ['Carry', 'Utility', 'Top', 'Jungle', 'Middle']
#     ranks = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER']

#     if request.method == 'POST':
#         position1 = request.form.get('position1')
#         position2 = request.form.get('position2')
#         rank = request.form.get('rank')

#         # Here is where you would call your function to calculate the recommended champions.
#         # I'm assuming it returns a list of champions.
#         recommended_champions = calculation(blue_champion, position1, position2, rank)

#         # This line is assuming that your calculation function also updates the unique_champions_list
#         # unique_champions_list = ...

#         return render_template_string(template, champions=unique_champions_list, remaining_champions=remaining_champions_list, blue_champion=blue_champion, recommended_champions=recommended_champions, positions=positions, remaining_positions=positions, ranks=ranks)

#     blue_side_champion = random.choice(unique_champions_list)
#     unique_champions_list.remove(blue_side_champion)

#     # Exclude the first position from the selection of the second position
#     remaining_positions = positions.copy()
#     remaining_positions.remove(positions[0])

#     return render_template_string(template, champions=unique_champions_list, remaining_champions=remaining_champions_list, blue_champion=blue_side_champion, positions=positions, remaining_positions=remaining_positions, ranks=ranks)

# # Run the Flask application in a separate thread
# def run_app():
#     app.run(port=5000)

# t = Thread(target=run_app)
# t.start()


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Jul/2023 15:07:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2023 15:07:54] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
# Import necessary modules
import pandas as pd
import random
from flask import Flask, render_template_string, request, redirect, url_for, session
from threading import Thread

# Create the Flask application
app = Flask(__name__)
app.config['SECRET_KEY'] = 'your secret key'  # replace 'your secret key' with your own secret key

# Assume 'League of Legends Champion Stats 13.3.csv' is your file containing champion names
df = pd.read_csv('League of Legends Champion Stats 13.3.csv', sep=';')
unique_champions_list = df['Name'].unique().tolist()

##Insert your algorithms here in the def calculation part. 

def calculation(blue_champion, position1, position2, rank): 
    # Randomly select two champions
    return random.sample(unique_champions_list, 2)

# Define the HTML template as a Python string
template = """
<!DOCTYPE html>
<html>
<head>
    <title>Select Your Champions</title>
</head>
<body>
    <div>
        <h2>Blue Side (Randomly Selected)</h2>
        <p>{{ blue_champion }}</p>
    </div>

    {% if not recommended_champions %}
    <div>
        <h2>Select Positions and Rank</h2>
        <form method="POST">
            <select name="position1">
                {% for position in positions %}
                    <option value="{{ position }}">{{ position }}</option>
                {% endfor %}
            </select>
            <select name="position2">
                {% for position in positions %}
                    <option value="{{ position }}">{{ position }}</option>
                {% endfor %}
            </select>
            <select name="rank">
                {% for rank in ranks %}
                    <option value="{{ rank }}">{{ rank }}</option>
                {% endfor %}
            </select>
            <input type="submit" value="Submit">
        </form>
    </div>
    {% endif %}

    {% if recommended_champions %}
    <div>
        <h2>Red Side (Select Your Champion)</h2>
        <form method="POST">
            <select name="chosen_champion">
                {% for champion in recommended_champions %}
                    <option value="{{ champion }}">{{ champion }}</option>
                {% endfor %}
            </select>
            <input type="submit" value="Submit">
        </form>
    {% endif %}
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def home():
    # Define positions and ranks
    positions = ['Carry', 'Utility', 'Top', 'Jungle', 'Middle']
    ranks = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER']

    # Select blue side champion
    if 'blue_champion' not in session:
        session['blue_champion'] = random.choice(unique_champions_list)
        unique_champions_list.remove(session['blue_champion'])

    if request.method == 'POST':
        if 'chosen_champion' in request.form:
            chosen_champion = request.form.get('chosen_champion')
            if chosen_champion in unique_champions_list:
                unique_champions_list.remove(chosen_champion)
            session.pop('recommended_champions', None)
        else:
            position1 = request.form.get('position1')
            position2 = request.form.get('position2')
            rank = request.form.get('rank')
            session['recommended_champions'] = calculation(session['blue_champion'], position1, position2, rank)

    return render_template_string(template, blue_champion=session['blue_champion'], recommended_champions=session.get('recommended_champions'), positions=positions, ranks=ranks)


In [2]:
# Run the Flask application in a separate thread
def run_app():
    app.run(port=5000)

t = Thread(target=run_app)
t.start()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jul/2023 16:03:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2023 16:03:16] "POST / HTTP/1.1" 200 -
